In [ ]:
import pandas as pd
from methods import clean_data

if __name__ == "__main__":
    # Load test data
    features = pd.read_parquet("test.parquet/date_id=0/part-0.parquet")
    responders = pd.read_parquet("lags.parquet/date_id=0/part-0.parquet")

    clean_features, clean_responders = clean_data(features, responders)
